In [17]:
import pandas as pd  # таблицы
import joblib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

dir = "D:\Develop\VS\HSE\Pump_Dump\\"  # папка проекта

In [18]:
main_df = pd.read_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|", index_col=0)
main_df.head()

,CHANNEL,NEWS_DATE,NEWS_TEXT,WORDS,TICKERS,WORDS_lemma,WAPRICE,Date,Price,Open,...,Change_A,Mood_0,Mood_1,Mood_2,Mood_B_0,Mood_B_1,Mood_B_2,Mood_A_0,Mood_A_1,Mood_A_2
0,if_stocks,2024-06-05,Российских инвесторов скоро засыплет дивиденда...,российских инвесторов скоро засыплет дивиденда...,NaN,российский инвестор скоро засыпать дивиденд тр...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,if_stocks,2024-06-05,20% 25%. Дивов не будет?! С 1 января 2025 года...,дивов января налог прибыль вырастет выиграет п...,NaN,див январь налог прибыль вырасти выиграть прои...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,if_stocks,2024-06-05,Рынки утром Коррекция на Мосбирже пошла на спа...,рынки утром коррекция мосбирже пошла спад каки...,"gazp,gaza",рынок утро коррекция мосбирже пойти спад какой...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,if_stocks,2024-06-04,Индекс Мосбиржи падает третью неделю. Мы дости...,индекс мосбиржи падает третью неделю достигли ...,posi,индекс мосбиржи падать третий неделя достигнут...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,if_stocks,2024-06-04,Информация для новичков Профессионалам на заме...,информация новичков профессионалам заметку кан...,"sber,astr,lkoh",информация новичок профессионал заметка канал ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
imoex_df = pd.read_csv(dir + "Answers\\imoex_changes.csv", encoding="utf-8", sep="|", index_col=0)
imoex_df.head()

,Date,Price,Open,High,Low,Change,Close,Before,After
1,2024-05-31,3212.40,3258.52,3269.97,3211.31,-2.06,3191.39,3280.11,3119.38
2,2024-05-30,3280.11,3332.77,3342.50,3275.54,-1.12,3295.44,3317.28,3212.40
3,2024-05-29,3317.28,3307.76,3323.08,3284.33,0.10,3311.07,3313.90,3280.11
4,2024-05-28,3313.90,3305.11,3347.76,3277.80,0.45,3319.98,3299.14,3317.28
5,2024-05-27,3299.14,3384.74,3392.99,3298.24,-2.91,3286.24,3398.11,3313.90


In [10]:
# проводим сентимент-анализ
# from textblob import TextBlob
# from textblob.sentiments import NaiveBayesAnalyzer # NLTK model trained on a movie reviews corpus

# def sentiment(text):
#     sent = TextBlob(text, analyzer=NaiveBayesAnalyzer())
#     return sent.sentiment

# main_df["Mood"] = main_df["WORDS_lemma"].apply(lambda x: sentiment(x))

In [4]:
# рассчитываем изменения цен до и после и категорируем их
# диапазоны изменения для категорирования взяты экспертно
borders = [-10, -0.6, 0.6, 10] # 0 - negative, 1 - neutral, 2 - positive

imoex_df["Change_B"] = round((imoex_df["Price"] - imoex_df["Before"])/imoex_df["Price"]*100,2)
imoex_df["Change_A"] = round((imoex_df["After"] - imoex_df["Price"])/imoex_df["Price"]*100,2)

imoex_df["Mood_B"] = pd.cut(imoex_df['Change_B'], borders).astype("category").cat.codes
imoex_df["Mood"] = pd.cut(imoex_df['Change'], borders).astype("category").cat.codes
imoex_df["Mood_A"] = pd.cut(imoex_df['Change_A'], borders).astype("category").cat.codes

imoex_df = pd.get_dummies(imoex_df, columns=["Mood"], dtype=int)
imoex_df = pd.get_dummies(imoex_df, columns=["Mood_B"], dtype=int)
imoex_df = pd.get_dummies(imoex_df, columns=["Mood_A"], dtype=int)
imoex_df.head()


,Date,Price,Open,High,Low,Change,Close,Before,After,Change_B,Change_A,Mood_0,Mood_1,Mood_2,Mood_B_0,Mood_B_1,Mood_B_2,Mood_A_0,Mood_A_1,Mood_A_2
1,2024-05-31,3212.40,3258.52,3269.97,3211.31,-2.06,3191.39,3280.11,3119.38,-2.11,-2.90,1,0,0,1,0,0,1,0,0
2,2024-05-30,3280.11,3332.77,3342.50,3275.54,-1.12,3295.44,3317.28,3212.40,-1.13,-2.06,1,0,0,1,0,0,1,0,0
3,2024-05-29,3317.28,3307.76,3323.08,3284.33,0.10,3311.07,3313.90,3280.11,0.10,-1.12,0,1,0,0,1,0,1,0,0
4,2024-05-28,3313.90,3305.11,3347.76,3277.80,0.45,3319.98,3299.14,3317.28,0.45,0.10,0,1,0,0,1,0,0,1,0
5,2024-05-27,3299.14,3384.74,3392.99,3298.24,-2.91,3286.24,3398.11,3313.90,-3.00,0.45,1,0,0,1,0,0,0,1,0


In [ ]:
# если полный файл был сформирован ранее, то используем его и новые данные не подгружаем IMOEX. При изменении границ категорирования файл надо обновить отдельно

# объединяем новости и цены ИНДЕКСА
# main_df = pd.merge(
#     main_df, imoex_df, left_on=["NEWS_DATE"], right_on=["Date"], how="left"
# )

# main_df.head()

In [20]:
# формируем фрейм для обучения модели
model_df = main_df[["WORDS_lemma", "Mood_0","Mood_1","Mood_2","Mood_B_0","Mood_B_1","Mood_B_2","Mood_A_0","Mood_A_1","Mood_A_2"]]
model_df.dropna(inplace = True)
model_df.head()


C:\Users\user\AppData\Local\Temp\ipykernel_10780\561888128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.dropna(inplace = True)


,WORDS_lemma,Mood_0,Mood_1,Mood_2,Mood_B_0,Mood_B_1,Mood_B_2,Mood_A_0,Mood_A_1,Mood_A_2
32,закрыться основный торговый сессия российский ...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
33,повышение налог ставка повлиять рынок экономик...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
34,говорить инвестор eutr gazp tcsg rosn nlmk под...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
35,ставка подключаться прямой эфир начаться обсуд...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
36,перестать терять деньга рынке начать зарабатыв...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# формируем обучающую и тестовые выборки
X = model_df["WORDS_lemma"].values.astype('U')
Y = model_df[model_df.columns[1:10]]

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((11318,), (11318, 9), (2830,), (2830, 9))

In [22]:
# проводим векторизацию. Закомментированы альтернативные векторайзеры

# vectorizer = CountVectorizer(ngram_range=(1, 3))
# X_train = vectorizer.fit_transform(X_train)
# X_test = vectorizer.transform(X_test)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [23]:
# Создание многозадачной модели с использованием случайного леса
multi_task_model = MultiOutputClassifier(RandomForestClassifier())

In [24]:
# Обучение модели на данных
multi_task_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [18]:
# Сохраняем обученную модель. !!! имя модели соответствует различным векторайзерам
# joblib.dumpa(multi_task_model, dir + "Answers\\model_classif_TFIDF.pkl") 

['D:\\Develop\\VS\\HSE\\Pump_Dump\\Answers\\model_classif_TFIDF.pkl']

In [25]:
# проводим предсказание и смотрим на качество работы модели

y_predict = multi_task_model.predict(X_test)
print("TFIDF_accuracy: ", accuracy_score(y_predict, y_test))
print(classification_report(y_test, y_predict))

TFIDF_accuracy:  0.3932862190812721
              precision    recall  f1-score   support

           0       0.38      0.01      0.02       434
           1       0.65      0.96      0.78      1821
           2       0.44      0.02      0.04       575
           3       0.27      0.01      0.01       434
           4       0.65      0.97      0.78      1821
           5       0.43      0.02      0.04       575
           6       0.40      0.01      0.03       415
           7       0.66      0.96      0.78      1846
           8       0.33      0.01      0.03       569

   micro avg       0.65      0.63      0.64      8490
   macro avg       0.47      0.33      0.28      8490
weighted avg       0.56      0.63      0.51      8490
 samples avg       0.65      0.63      0.63      8490



c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
# сохраняем собранный фрейм на диск
# main_df.to_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|")

In [ ]:
# Загружаем обученную модель. !!! имя модели соответсвует различным векторайзерам
# multi_task_model = joblib.load("Answers\\model_classif_TFIDF.pkl")